In [1]:
import pandas as pd
import numpy as np

import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    import nltk
    from nltk.tokenize import RegexpTokenizer
    from nltk.stem.wordnet import WordNetLemmatizer
    from nltk.corpus import stopwords
    from nltk import FreqDist

    import gensim
    from gensim import corpora, models, similarities
    import pyLDAvis
    import pyLDAvis.gensim

/home/sam/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/home/sam/anaconda3/envs/insight/lib/python3.7/site-packages/funcy/colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
/home/sam/anaconda3/envs/insight/lib/python3.7/site-packages/funcy/colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
/home/sam/anaconda3/envs/insight/lib/python3.7/site-packages/funcy/flow.py:2: DeprecationWarning: Using or importing the ABCs fro

In [2]:
cities_raw = pd.read_csv('cities_geo2_df.csv', index_col=0)
cities = cities_raw.loc[:,:'Get out'] # Just text portions of the df

# Reset index
cities = cities.reset_index()
cities.drop(['index'], axis=1, inplace=True)

In [3]:
cities.head()

,Country,City,Understand,Get in,Get around,See,Do,Buy,Eat,Sleep,Cope,Stay safe,Get out
0,Afghanistan,Herat,Herat is the second largest city in Afghanista...,The Herat International Airport is situated 15...,NaN,"Ghala Ekhteyaradin, Takht Safar, Bagh Milat, ...",Take a shower in the huge subterranean Hammams.,There are several antique shops on the north ...,Herat has a mixture of traditional and modern ...,"Budget[edit] Mowafaq Hotel, northeast corner ...",NaN,Herat is one of the safer cities in Afghanista...,Chisht-i-Sharif is some 177 km from Herat city...
1,Afghanistan,Kabul,Kabul is a very historic city of the region th...,By plane[edit] Kabul International Airport (IA...,Maps of Kabul are available from Afghanistan I...,Bagh-e Babur (Gardens of Babur). The gardens ...,Kabul Wall. A pleasant hike with rewarding vi...,The Share-e Naw area has some shops. The Kab...,The once thriving restaurant scene in Kabul ha...,"Kabul is not a cheap place to stay, principall...",Read the Scene magazine for restaurant reviews...,Kabul is generally considered one of the safer...,Most expats take any opportunity they can to l...
2,Afghanistan,Jalalabad,NaN,Jalalabad is on the major highway that links K...,NaN,The large reservoir near Sarobi. This is loca...,Go fishing along the river side. Go swimming .,Handi Craft Handicraft is one of most famous ...,Pakora go for special “Pakora” Food in Hindu ...,The government hotel Spingar is on the easter...,NaN,NaN,This article is an outline and needs more co...
3,Afghanistan,Kandahar,Kandahār or Qandahār (Pashto: کندهار ) (Persia...,By plane[edit] Kandahar International Airport ...,NaN,"Kandahar Museum, (Western end of the Eidgah ...",NaN,"Afghanistan International Bank (AIB), Kabul Ba...","There are plenty of food choices in Kandahar, ...","Budget[edit] Armani Hotel, (Two miles from c...",NaN,"As of 2013, the city centre of Kandahar is qui...",Kabul Helmand Harat Zabul Spin Boldak (Spin Bu...
4,Afghanistan,Kunduz,NaN,Most visitors will arrive by road from Pol-e K...,"The city is fairly small. The local roads, li...",There's not much for tourists here.,NaN,NaN,The usual Afghan fare - kebab or lamb with ric...,NaN,NaN,NaN,This article is an outline and needs more ...


In [4]:
# Run once
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')

In [5]:
tokenizer = RegexpTokenizer(r'\w+') # retains alphanumeric character
lmtzr = WordNetLemmatizer()
cached_stop_words = stopwords.words("english")

In [6]:
def lemmatize_dropstop_words(text): 
    return [lmtzr.lemmatize(word) for word in text if lmtzr.lemmatize(word) not in cached_stop_words]

In [7]:
cities = cities.replace(np.nan, '', regex=True)

In [8]:
cities.head()

,Country,City,Understand,Get in,Get around,See,Do,Buy,Eat,Sleep,Cope,Stay safe,Get out
0,Afghanistan,Herat,Herat is the second largest city in Afghanista...,The Herat International Airport is situated 15...,,"Ghala Ekhteyaradin, Takht Safar, Bagh Milat, ...",Take a shower in the huge subterranean Hammams.,There are several antique shops on the north ...,Herat has a mixture of traditional and modern ...,"Budget[edit] Mowafaq Hotel, northeast corner ...",,Herat is one of the safer cities in Afghanista...,Chisht-i-Sharif is some 177 km from Herat city...
1,Afghanistan,Kabul,Kabul is a very historic city of the region th...,By plane[edit] Kabul International Airport (IA...,Maps of Kabul are available from Afghanistan I...,Bagh-e Babur (Gardens of Babur). The gardens ...,Kabul Wall. A pleasant hike with rewarding vi...,The Share-e Naw area has some shops. The Kab...,The once thriving restaurant scene in Kabul ha...,"Kabul is not a cheap place to stay, principall...",Read the Scene magazine for restaurant reviews...,Kabul is generally considered one of the safer...,Most expats take any opportunity they can to l...
2,Afghanistan,Jalalabad,,Jalalabad is on the major highway that links K...,,The large reservoir near Sarobi. This is loca...,Go fishing along the river side. Go swimming .,Handi Craft Handicraft is one of most famous ...,Pakora go for special “Pakora” Food in Hindu ...,The government hotel Spingar is on the easter...,,,This article is an outline and needs more co...
3,Afghanistan,Kandahar,Kandahār or Qandahār (Pashto: کندهار ) (Persia...,By plane[edit] Kandahar International Airport ...,,"Kandahar Museum, (Western end of the Eidgah ...",,"Afghanistan International Bank (AIB), Kabul Ba...","There are plenty of food choices in Kandahar, ...","Budget[edit] Armani Hotel, (Two miles from c...",,"As of 2013, the city centre of Kandahar is qui...",Kabul Helmand Harat Zabul Spin Boldak (Spin Bu...
4,Afghanistan,Kunduz,,Most visitors will arrive by road from Pol-e K...,"The city is fairly small. The local roads, li...",There's not much for tourists here.,,,The usual Afghan fare - kebab or lamb with ric...,,,,This article is an outline and needs more ...


In [9]:
# Tokenize, lemmatize
cities['Understand_tokens'] = cities['Understand'].str.lower().apply(tokenizer.tokenize).apply(lemmatize_dropstop_words)
cities['See_tokens'] = cities['See'].str.lower().apply(tokenizer.tokenize).apply(lemmatize_dropstop_words)
cities['Do_tokens'] = cities['Do'].str.lower().apply(tokenizer.tokenize).apply(lemmatize_dropstop_words)
cities['Buy_tokens'] = cities['Buy'].str.lower().apply(tokenizer.tokenize).apply(lemmatize_dropstop_words)
cities['Eat_tokens'] = cities['Eat'].str.lower().apply(tokenizer.tokenize).apply(lemmatize_dropstop_words)

In [10]:
cities.head()

,Country,City,Understand,Get in,Get around,See,Do,Buy,Eat,Sleep,Cope,Stay safe,Get out,Understand_tokens,See_tokens,Do_tokens,Buy_tokens,Eat_tokens
0,Afghanistan,Herat,Herat is the second largest city in Afghanista...,The Herat International Airport is situated 15...,,"Ghala Ekhteyaradin, Takht Safar, Bagh Milat, ...",Take a shower in the huge subterranean Hammams.,There are several antique shops on the north ...,Herat has a mixture of traditional and modern ...,"Budget[edit] Mowafaq Hotel, northeast corner ...",,Herat is one of the safer cities in Afghanista...,Chisht-i-Sharif is some 177 km from Herat city...,"[herat, second, largest, city, afghanistan, lo...","[ghala, ekhteyaradin, takht, safar, bagh, mila...","[take, shower, huge, subterranean, hammams]","[several, antique, shop, north, side, mosque, ...","[herat, ha, mixture, traditional, modern, food..."
1,Afghanistan,Kabul,Kabul is a very historic city of the region th...,By plane[edit] Kabul International Airport (IA...,Maps of Kabul are available from Afghanistan I...,Bagh-e Babur (Gardens of Babur). The gardens ...,Kabul Wall. A pleasant hike with rewarding vi...,The Share-e Naw area has some shops. The Kab...,The once thriving restaurant scene in Kabul ha...,"Kabul is not a cheap place to stay, principall...",Read the Scene magazine for restaurant reviews...,Kabul is generally considered one of the safer...,Most expats take any opportunity they can to l...,"[kabul, historic, city, region, wa, built, alm...","[bagh, e, babur, garden, babur, garden, surrou...","[kabul, wall, pleasant, hike, rewarding, view,...","[share, e, naw, area, ha, shop, kabul, city, c...","[thriving, restaurant, scene, kabul, ha, suffe..."
2,Afghanistan,Jalalabad,,Jalalabad is on the major highway that links K...,,The large reservoir near Sarobi. This is loca...,Go fishing along the river side. Go swimming .,Handi Craft Handicraft is one of most famous ...,Pakora go for special “Pakora” Food in Hindu ...,The government hotel Spingar is on the easter...,,,This article is an outline and needs more co...,[],"[large, reservoir, near, sarobi, located, way,...","[go, fishing, along, river, side, go, swimming]","[handi, craft, handicraft, one, famous, produc...","[pakora, go, special, pakora, food, hindu, str..."
3,Afghanistan,Kandahar,Kandahār or Qandahār (Pashto: کندهار ) (Persia...,By plane[edit] Kandahar International Airport ...,,"Kandahar Museum, (Western end of the Eidgah ...",,"Afghanistan International Bank (AIB), Kabul Ba...","There are plenty of food choices in Kandahar, ...","Budget[edit] Armani Hotel, (Two miles from c...",,"As of 2013, the city centre of Kandahar is qui...",Kabul Helmand Harat Zabul Spin Boldak (Spin Bu...,"[kandahār, qandahār, pashto, کندهار, persian, ...","[kandahar, museum, western, end, eidgah, durwa...",[],"[afghanistan, international, bank, aib, kabul,...","[plenty, food, choice, kandahar, however, serf..."
4,Afghanistan,Kunduz,,Most visitors will arrive by road from Pol-e K...,"The city is fairly small. The local roads, li...",There's not much for tourists here.,,,The usual Afghan fare - kebab or lamb with ric...,,,,This article is an outline and needs more ...,[],"[much, tourist]",[],[],"[usual, afghan, fare, kebab, lamb, rice, river..."


In [11]:
# Most common words
words = []
for sublist in cities['See_tokens']:
    for item in sublist:
        words.append(item)
        
fdist = FreqDist(words)
fdist.most_common(50)

[('museum', 9761),
 ('edit', 9639),
 ('city', 6420),
 ('park', 5681),
 ('wa', 5456),
 ('one', 4413),
 ('ha', 4158),
 ('also', 3997),
 ('art', 3744),
 ('building', 3690),
 ('temple', 3563),
 ('1', 3229),
 ('located', 2933),
 ('house', 2858),
 ('church', 2801),
 ('built', 2800),
 ('area', 2800),
 ('street', 2716),
 ('old', 2699),
 ('place', 2636),
 ('free', 2614),
 ('garden', 2377),
 ('00', 2338),
 ('many', 2325),
 ('see', 2259),
 ('century', 2224),
 ('2', 2122),
 ('well', 2083),
 ('bus', 2076),
 ('5', 2068),
 ('take', 2055),
 ('town', 2025),
 ('de', 1998),
 ('st', 1996),
 ('10', 1980),
 ('open', 1970),
 ('world', 1966),
 ('year', 1901),
 ('around', 1900),
 ('center', 1879),
 ('history', 1827),
 ('station', 1804),
 ('view', 1753),
 ('day', 1751),
 ('road', 1743),
 ('collection', 1741),
 ('small', 1738),
 ('3', 1730),
 ('square', 1713),
 ('tour', 1700)]

In [12]:
# NLP

# Create dictionaries of unique words in the processed tokenized text
See_dict = corpora.Dictionary(cities['See_tokens'])

In [13]:
# Convert texts to vectors
See_corpus = [See_dict.doc2bow(text) for text in cities['See_tokens']]

In [14]:
# LDA
See_lda = gensim.models.ldamodel.LdaModel(corpus=See_corpus,
                                               id2word=See_dict,
                                               num_topics=30, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=1000,
                                               passes=1,
                                               alpha='auto',
                                               eta='auto')

In [15]:
# Visualize LDA results
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(See_lda, See_corpus, See_dict)
vis

/home/sam/anaconda3/envs/insight/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
26    -0.092683 -0.001229       1        1  19.825626
9     -0.087490  0.023060       2        1  17.072458
13    -0.097080 -0.016527       3        1   8.811743
5     -0.088533  0.009744       4        1   6.760370
29    -0.063260  0.033047       5        1   6.320532
19    -0.017888  0.069196       6        1   5.595939
14    -0.086546  0.026248       7        1   5.312099
27    -0.089585  0.026112       8        1   4.079388
8     -0.035507 -0.013396       9        1   3.639859
12    -0.030942 -0.070866      10        1   3.145347
20    -0.027889  0.015950      11        1   3.091929
28     0.096661  0.101352      12        1   2.894327
21    -0.006750 -0.020154      13        1   1.661984
24    -0.001138 -0.040136      14        1   1.646933
7     -0.006915  0.025796      15        1   1.272505
0     -0.006439 -0.057535      16        1   1.202803
1      0.010104 -0.038161      17        1   1.187908
23     0.001575 -0.014786      18        1   0.877581
22     0.013611 -0.016080      19        1   0.791109
18     0.021420 -0.016737      20        1   0.694242
6      0.032622 -0.009863      21        1   0.553864
15     0.037622 -0.002986      22        1   0.488801
10     0.040719  0.011195      23        1   0.479076
2      0.074483  0.032140      24        1   0.460227
16     0.041786  0.019891      25        1   0.458616
4      0.071820  0.010159      26        1   0.435127
3      0.075592 -0.044716      27        1   0.407611
17     0.066556 -0.008800      28        1   0.359228
11     0.057250 -0.019536      29        1   0.297924
25     0.096823 -0.012383      30        1   0.174846, topic_info=      Category         Freq       Term        Total  loglift  logprob
term                                                                 
139    Default  9156.000000     museum  9156.000000  30.0000  30.0000
62     Default  8484.000000       edit  8484.000000  29.0000  29.0000
42     Default  5593.000000       city  5593.000000  28.0000  28.0000
158    Default  5151.000000       park  5151.000000  27.0000  27.0000
227    Default  5131.000000         wa  5131.000000  26.0000  26.0000
4436   Default  2673.000000     temple  2673.000000  25.0000  25.0000
151    Default  4010.000000        one  4010.000000  24.0000  24.0000
33     Default  3499.000000   building  3499.000000  23.0000  23.0000
87     Default  3778.000000         ha  3778.000000  22.0000  22.0000
278    Default  3472.000000       also  3472.000000  21.0000  21.0000
1111   Default  2188.000000     church  2188.000000  20.0000  20.0000
689    Default  3936.000000        art  3936.000000  19.0000  19.0000
623    Default  2153.000000         00  2153.000000  18.0000  18.0000
0      Default  3760.000000          1  3760.000000  17.0000  17.0000
121    Default  2611.000000    located  2611.000000  16.0000  16.0000
1027   Default  3104.000000     street  3104.000000  15.0000  15.0000
376    Default  2786.000000      house  2786.000000  14.0000  14.0000
39     Default  1976.000000    century  1976.000000  13.0000  13.0000
150    Default  2383.000000        old  2383.000000  12.0000  12.0000
287    Default  2374.000000       area  2374.000000  11.0000  11.0000
1144   Default  1500.000000         de  1500.000000  10.0000  10.0000
34     Default  2517.000000      built  2517.000000   9.0000   9.0000
1422   Default  2176.000000         st  2176.000000   8.0000   8.0000
163    Default  2226.000000      place  2226.000000   7.0000   7.0000
716    Default  1546.000000        bus  1546.000000   6.0000   6.0000
75     Default  2825.000000       free  2825.000000   5.0000   5.0000
7      Default  1939.000000          5  1939.000000   4.0000   4.0000
126    Default  2139.000000       many  2139.000000   3.0000   3.0000
216    Default  1750.000000       town  1750.000000   2.0000   2.0000
240    Default  1825.000000         10  1825.000000   1.000

In [16]:
# Recommendation

# Create vectors that represent topic distribution of each lyric or comment
See_topic_vectors = See_lda[See_corpus]

In [17]:
# Search cities

cities[cities['City'] == 'Beijing']

,Country,City,Understand,Get in,Get around,See,Do,Buy,Eat,Sleep,Cope,Stay safe,Get out,Understand_tokens,See_tokens,Do_tokens,Buy_tokens,Eat_tokens
321,China,Beijing,History[edit] Beijing literally means Northern...,By plane[edit] Beijing is generally served by ...,A note on maps Beijing is changing at such...,See the Districts articles for individual list...,See the Districts articles for individual list...,See the Districts articles for individual list...,See the Districts articles for individual list...,See the Districts articles for individual list...,Good bilingual maps are hard to find in Beijin...,"Crime[edit] Overall, Beijing is a very safe ci...",Long distance cyclist-tourists will find natio...,"[history, edit, beijing, literally, mean, nort...","[see, district, article, individual, listing, ...","[see, district, article, individual, listing, ...","[see, district, article, individual, listing, ...","[see, district, article, individual, listing, ..."


In [18]:
input_city = 321

In [19]:
See_sims = []
for i in np.arange(len(See_topic_vectors)):
    sim = gensim.matutils.cossim(See_topic_vectors[input_city], See_topic_vectors[i])
    See_sims.append(sim)
cities['Similarity to input'] = See_sims

In [20]:
cities.sort_values(by=['Similarity to input'], ascending = False).head(10)

,Country,City,Understand,Get in,Get around,See,Do,Buy,Eat,Sleep,Cope,Stay safe,Get out,Understand_tokens,See_tokens,Do_tokens,Buy_tokens,Eat_tokens,Similarity to input
321,China,Beijing,History[edit] Beijing literally means Northern...,By plane[edit] Beijing is generally served by ...,A note on maps Beijing is changing at such...,See the Districts articles for individual list...,See the Districts articles for individual list...,See the Districts articles for individual list...,See the Districts articles for individual list...,See the Districts articles for individual list...,Good bilingual maps are hard to find in Beijin...,"Crime[edit] Overall, Beijing is a very safe ci...",Long distance cyclist-tourists will find natio...,"[history, edit, beijing, literally, mean, nort...","[see, district, article, individual, listing, ...","[see, district, article, individual, listing, ...","[see, district, article, individual, listing, ...","[see, district, article, individual, listing, ...",0.999994
1076,India,Patan,,"Micro Bus from Ratnapark in Kathmandu (12 rps,...",The city and its sites are best seen on foot. ...,Durbar Square[edit] NOTICE: Many of the temple...,Just wander around keeping your eyes open for ...,Wooden and Metal Handicrafts. There are many m...,"Cafe de Patan - a pleasant, centraly located r...",While most tourists stay in hotels in Kathmand...,,,Kirtipur is one of the oldest settlements in ...,[],"[durbar, square, edit, notice, many, temple, l...","[wander, around, keeping, eye, open, amazing, ...","[wooden, metal, handicraft, many, metalworking...","[cafe, de, patan, pleasant, centraly, located,...",0.989097
1122,India,Thiruvananthapuram,Sanskrit History[edit] Thiruvananthapuram is...,By plane[edit] Trivandrum International Airpor...,By bus[edit] Malayalam East Fort Thampanoor By...,"Thiruvananthapuram is a historic city, dotted ...",Stroll around the town during the dusk time w...,Haggling is quite acceptable and very much nee...,This guide uses the following price ranges ...,Unless you are in Trivandrum on a business tri...,Tourist Information[edit] There is a tourist i...,Trivandrum is generally a safe city.Although y...,Ponmudi - A pleasant hill resort 61 km away f...,"[sanskrit, history, edit, thiruvananthapuram, ...","[thiruvananthapuram, historic, city, dotted, m...","[stroll, around, town, dusk, time, city, come,...","[haggling, quite, acceptable, much, needed, bu...","[guide, us, following, price, range, typical, ...",0.986906
320,China,Anyang,Anyang is 460 km south of Beijing and is a pre...,"By bus or train. From Beijing, about 3.5 hour...","Bus Y1 runs between Anyang East Train Station,...","Yin Ruins (﻿殷墟; Yīnxū), (taxi fare from down...",,,,"Super 8 Hotel (速8酒店), (next to the ticket ha...",,,This article is an outline and needs more ...,"[anyang, 460, km, south, beijing, prefecture, ...","[yin, ruin, 殷墟, yīnxū, taxi, fare, downtown, 1...",[],[],[],0.968836
678,Côte d'Ivoire,Yamoussoukro,,Every UOB bus from Abijan to Bouake or Man sto...,,"The public gardens, the Fondation Felix Houpho...",,,Chez Georges Hollywood. The Chez Georges Holly...,"Hotel President. edit Hotel Villa des Hautes,...",,,This article is an outline and needs more co...,[],"[public, garden, fondation, felix, houphouet, ...",[],[],"[chez, george, hollywood, chez, george, hollyw...",0.968624
1034,India,Kottayam,"Kottayam is a town in south-central Kerala, sa...",By plane[edit] Kochi International Airport [4]...,Bus - Kottayam has a comprehenhsive local bus...,"Aruvikkuzhi Waterfalls, (18 km from Kottayam...",Boat Races. Lots of boat races with tradition...,,There are numerous restaurants and cafes in Ko...,There are a number of hotels of varying qualit...,,,,"[kottayam, town, south, central, kerala, sandw...","[aruvikkuzhi, waterfall, 18, km, kottayam, tow...","[boat, race, lot, boat, race, traditional, boa...",[],"[numerous, restaurant, cafe, kottayam, town, l...",0.964685
854,Ghana,Kumasi,Kumasi is considered the home of the Ashanti K